In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')

import re
import os

# IONOMER

## .mol headers:

* total atoms
* total bonds
* total angles
* total dihedrals

* coords
* types ∀ atoms
* charges ∀ atoms
* bonds ∀ atoms
* angles ∀ atoms
* dihedrals ∀ atoms

GROMOS FF: 4377 lines

DREIDING FF: 4439 lines

In [2]:
with open("../relaxed_ionomer_.data", "r") as file:
    file_content = file.readlines()
# Create a DataFrame with each line as a separate row in a single column
data = pd.DataFrame(file_content, columns=['data'])    

In [3]:
coords = data.loc[88:585].copy().reset_index(drop=True)
coords["atom id"] = coords["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
coords["molecule id"] = coords["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
coords["atom type"] = coords["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
coords["charge"] = coords["data"].str.split('\s').apply(lambda x: x[3]).astype(float)
coords["x"] = coords["data"].str.split('\s').apply(lambda x: x[4]).astype(float)
coords["y"] = coords["data"].str.split('\s').apply(lambda x: x[5]).astype(float)
coords["z"] = coords["data"].str.split('\s').apply(lambda x: x[6]).astype(float)
coords = coords.sort_values("atom id").reset_index(drop=True)
coords["element"] = coords["atom type"].map({1:"F",2:"C",3:"O_hyd",
                                             4:"O",5:"S",6:"H"})
coords["element"].unique()

array(['F', 'C', 'O_hyd', 'O', 'S'], dtype=object)

In [4]:
bonds = data.loc[1090:1594].copy().reset_index(drop=True)
bonds["bond id"] = bonds["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
bonds["bond type"] = bonds["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
bonds["atom1"] = bonds["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
bonds["atom2"] = bonds["data"].str.split('\s').apply(lambda x: x[3]).astype(int)

In [5]:
angles = data.loc[1598:2581].copy().reset_index(drop=True)
angles["angle id"] = angles["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
angles["angle type"] = angles["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
angles["atom1"] = angles["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
angles["atom2"] = angles["data"].str.split('\s').apply(lambda x: x[3]).astype(int)
angles["atom3"] = angles["data"].str.split('\s').apply(lambda x: x[4]).astype(int)

In [6]:
dihedrals = data.loc[2585:].copy().reset_index(drop=True)
dihedrals["dihedral id"] = dihedrals["data"].str.split('\s').apply(lambda x: x[0]).astype(int)
dihedrals["dihedral type"] = dihedrals["data"].str.split('\s').apply(lambda x: x[1]).astype(int)
dihedrals["atom1"] = dihedrals["data"].str.split('\s').apply(lambda x: x[2]).astype(int)
dihedrals["atom2"] = dihedrals["data"].str.split('\s').apply(lambda x: x[3]).astype(int)
dihedrals["atom3"] = dihedrals["data"].str.split('\s').apply(lambda x: x[4]).astype(int)
dihedrals["atom4"] = dihedrals["data"].str.split('\s').apply(lambda x: x[5]).astype(int)

In [7]:
def write_mol_file():
    f = open("nafion_ionomer_dreiding.mol", "w")
    f.write("# Created by Karina Chiñas Fuentes | Python | 07.08.24\n\n")
    f.write("\n506 atoms\n505 bonds\n984 angles\n1407 dihedrals")
    f.write("\n\n")
    f.write("Coords\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['x'][i]}\t{coords['y'][i]}\t{coords['z'][i]}\n")
    f.write("\nTypes\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['atom type'][i]}\n")
    f.write("\nCharges\n\n")
    for i in range(len(coords)):
        f.write(f"{coords['atom id'][i]}\t{coords['charge'][i]}\n")
    f.write("\nBonds\n\n")
    for i in range(len(bonds)):
        f.write(f"{bonds['bond id'][i]}\t{bonds['bond type'][i]}\t"+
                f"{bonds['atom1'][i]}\t{bonds['atom2'][i]}\n")
    f.write("\nAngles\n\n")
    for i in range(len(angles)):
        f.write(f"{angles['angle id'][i]}\t{angles['angle type'][i]}\t"+
                f"{angles['atom1'][i]}\t{angles['atom2'][i]}\t{angles['atom3'][i]}\n")
    f.write("\nDihedrals\n\n")
    for i in range(len(dihedrals)):
        f.write(f"{dihedrals['dihedral id'][i]}\t{dihedrals['dihedral type'][i]}\t{dihedrals['atom1'][i]}\t"+
                f"{dihedrals['atom2'][i]}\t{dihedrals['atom3'][i]}\t{dihedrals['atom4'][i]}\n")

In [8]:
#write_mol_file()

## parm.lammps headers:

* mass ∀ types
* pair_coeff ∀ types (individual and crossed)
* bond_coeff ∀ types
* angle_coeff ∀ types
* dihedral_coeff ∀ types

(Did it by hand)